In [11]:
import pandas as pd

# load data
weapons = pd.read_csv('mhw_weapon_properties.csv', index_col='Name')
weapon_types = pd.read_csv('mhw_weapon_type_properties.csv', index_col='Weapon Type')
weapon_sharpness = pd.read_csv('mhw_sharpness_damage_multipliers.csv', index_col='Damage Type')
monsters = pd.read_csv('mhw_monster_damage_multipliers.csv', index_col='Monster')

# Weapon type motion values are mean from all values at https://www.reddit.com/r/MonsterHunter/comments/7v0pp3/mhworld_motion_values_compiled/
# Weapon type average hit per second are from https://www.reddit.com/r/MonsterHunter/comments/4e9o13/weapons_by_attack_speed/
# Monster damage modifiers are average values from https://honeyhunterworld.com/mhwb/
# Weapon properties are from https://monsterhunterworld.wiki.fextralife.com/Weapons

modifier_data = []

In [12]:
import itertools as it

# armor and buff modifiers
affinity_modifier = 0
attack_boost = 0
attack_modifier = 0
elemental_boost = 0
elemental_modifier = 0
critical_element = 0
handicraft = 0

# calculate DPS for weapons and monsters
data = []
selected_weapons = ["Daora's Tethidine", 'Firedance Rathmaul', 'Grunge Storm', 'Tyrannis Glaive II', 'Vice']
for m, w in it.product(monsters.index, selected_weapons):
    t = weapons.loc[w, 'Type']
    if handicraft == 1:
        s = weapons.loc[w, 'Handicraft Sharpness']
    else:
        s = weapons.loc[w, 'Sharpness']    
    raw_dps = ((weapons.loc[w, 'Damage'] / weapon_types.loc[t, 'Multiplier']) + attack_boost) * \
              ((attack_modifier / 100) + 1) * \
              weapon_sharpness.loc['Raw', s] * \
              (((weapons.loc[w, 'Affinity'] + affinity_modifier) / 400) + 1) * \
              (weapon_types.loc[t, 'Average Motion Multiplier'] / 100) * \
              (monsters.loc[m, weapons.loc[w, 'Raw Type']] / 100) * \
              weapon_types.loc[t, 'Average Hits Per Second']
    elem_dps = ((weapons.loc[w, 'Elemental Damage'] + elemental_boost) / 10) * \
               ((elemental_modifier / 100) + 1) * \
               weapon_sharpness.loc['Elemental', s] * \
               ((((weapons.loc[w, 'Affinity'] + affinity_modifier) / 400) * critical_element) + 1) * \
               (monsters.loc[m, weapons.loc[w, 'Elemental Type']] / 100) * \
               weapon_types.loc[t, 'Average Hits Per Second']
    data.append([w, m, raw_dps, elem_dps, raw_dps + elem_dps])

dmg_df = pd.DataFrame(data, columns=['Weapon', 'Against', 'Raw DPS', 'Elem DPS', 'Total DPS'])
modifier_data.append([affinity_modifier, attack_boost, attack_modifier, elemental_boost, elemental_modifier, critical_element, handicraft, dmg_df['Total DPS'].sum()])
dmg_df
#dmg_df.groupby(['Weapon']).mean()

,Raw DPS,Elem DPS,Total DPS
Weapon,,,
Daora's Tethidine,38.384413,5.318025,43.702438
Firedance Rathmaul,41.505097,2.514725,44.019822
Grunge Storm,42.909405,4.136242,47.045647
Tyrannis Glaive II,44.447936,0.000000,44.447936
Vice,38.446827,0.000000,38.446827
